# Advent of Code 2023
## Day 23
*<https://adventofcode.com/2023/day/23>*

In [1]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [2]:
DAY = 23
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [3]:
inp = input_str.parse_grid()

In [4]:
class Region:
    positions: set[tuple[int, int]]
    neighbours: set["Region"]

    def __init__(self):
        self.positions = set()
        self.neighbours = set()

    def add(self, x, y):
        self.positions.add((x, y))

    def add_neighbour(self, other: "Region"):
        self.neighbours.add(other)

    def __contains__(self, item):
        return item in self.positions

    def size(self):
        return len(self.positions)

    def __repr__(self):
        return f"Region({self.size()})"


regions: list[Region] = []

for (x, y), c in inp.items():
    if c == ".":
        if any((x, y) in r for r in regions):
            continue

        r = Region()
        regions.append(r)
        r.add(x, y)

        q = deque([(x, y)])
        while q:
            x, y = q.popleft()
            for (nx, ny), nc in inp.adjacent_4_items(x, y):
                if nc == "." and (nx, ny) not in r:
                    r.add(nx, ny)
                    q.append((nx, ny))

In [5]:
def get_region(x, y):
    for r in regions:
        if (x, y) in r:
            return r

    assert False, "no region found"

In [6]:
sx, sy = inp.find(".")
assert sy == 0

DIRS = {
    ">": (1, 0),
    "<": (-1, 0),
    "^": (0, -1),
    "v": (0, 1),
}

start_reg: Region = [r for r in regions if (sx, sy) in r][0]
end_reg: Region = [r for r in regions if (inp.width - 2, inp.height - 1) in r][0]

for (x, y), c in inp.items():
    if c in "<>v^":
        dx, dy = DIRS[c]
        prev = get_region(x - dx, y - dy)
        next = get_region(x + dx, y + dy)

        prev.add_neighbour(next)

Could have got top 50 for part 2 here if I'd realised that while using `0` as a base value for part 1 works fine, for part 2 it does not.

In [7]:
def longest_path_regions(cur: Region, end: Region, visited: set[Region] = None):
    if cur == end:
        return end_reg.size()

    if visited is None:
        visited = set()

    visited.add(cur)

    longest = -math.inf
    for n in cur.neighbours:
        if n in visited:
            continue

        longest = max(longest, 1 + cur.size() + longest_path_regions(n, end, visited))

    visited.remove(cur)
    return longest


part_1 = longest_path_regions(start_reg, end_reg) - 1

In [8]:
for r in regions:
    for n in r.neighbours:
        n.add_neighbour(r)

part_2 = longest_path_regions(start_reg, end_reg) - 1

In [9]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 2430

Part 2: 6534